In [ ]:

import numpy as np
import random
from tqdm.auto import tqdm

problem = np.load('data/problem_2.npz')

x_data = problem['x']
y_data = problem['y']

num_variables=x_data.shape[0]


BINARY_FUNCTIONS = [
    np.add, 
    np.subtract, 
    np.multiply, 
    np.divide, 
    np.power
]
UNARY_FUNCTIONS = [
    np.sin, 
    np.cos, 
    np.tan, 
    np.exp, 
    np.log,
    np.sqrt
]

variables = ["x0", "x1", "x2", "x3", "x4","x5",] 

variable_used = variables[:num_variables]

constants = list(range(-9, 10)) #da -9 a 9
TERMINALS = variable_used + constants
random.seed(42)


FUNCTIONS = BINARY_FUNCTIONS + UNARY_FUNCTIONS


class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def evaluate(self, x_data):
        if callable(self.value):
            left_val = self.left.evaluate(x_data) if self.left else None
            right_val = self.right.evaluate(x_data) if self.right else None

            try:
                # Gestione degli errori matematici
                if self.value == np.log and (left_val is not None and np.any(left_val <= 0)):
                    return np.full(x_data.shape[1], np.inf)
                if self.value == np.sqrt and (left_val is not None and np.any(left_val < 0)):
                    return np.full(x_data.shape[1], np.inf)
                if self.value == np.divide and (right_val is not None and np.any(right_val == 0)):
                    return np.full(x_data.shape[1], np.inf)

            # controlli per esplosione\overflow ( provali : )
                '''   provaaa '''

                # Gestione specifica per np.exp: clip dell'input
                if self.value == np.exp:
                    safe_input = np.clip(left_val, -100, 100)
                    result = np.exp(safe_input)
                    if np.any(~np.isfinite(result)):
                        return np.full(x_data.shape[1], np.inf)
                    return result

                # Gestione specifica per np.power: clip di base ed esponente
                if self.value == np.power:
                    safe_left = np.clip(left_val, -100, 100)
                    safe_right = np.clip(right_val, -10, 10)
                    result = np.power(safe_left, safe_right)
                    if np.any(~np.isfinite(result)):
                        return np.full(x_data.shape[1], np.inf)
                    return result

                # Operazioni standard per le funzioni unarie e binarie
                if self.value in UNARY_FUNCTIONS:
                    result = self.value(left_val)
                    if np.any(~np.isfinite(result)):
                        return np.full(x_data.shape[1], np.inf)
                    return result
                elif self.value in BINARY_FUNCTIONS:
                    result = self.value(left_val, right_val)
                    if np.any(~np.isfinite(result)):
                        return np.full(x_data.shape[1], np.inf)
                    return result

            except:
                return np.full(x_data.shape[1], np.inf)

        elif isinstance(self.value, str):
            # Variabili ("x", "z", "w")
            if self.value == "x0":
                return x_data[0, :]  
            elif self.value == "x1":
                return x_data[1, :]  
            elif self.value == "x2":
                return x_data[2, :] 
            elif self.value == "x3":
                return x_data[3, :]  
            elif self.value == "x4":
                return x_data[4, :]  
            elif self.value == "x5":
                return x_data[5, :]  
            
        else:
            # Terminale numerico, restituisci un array costante
            return np.full(x_data.shape[1], self.value)

        raise ValueError("Nodo non valido")


    def __str__(self):
     function_symbols = {
        np.add: "+",
        np.subtract: "-",
        np.multiply: "*",
        np.divide: ":",
        np.power: "^"
     }

     if callable(self.value):
        if self.value in UNARY_FUNCTIONS:
            return f"{self.value.__name__}({self.left})"
        elif self.value in BINARY_FUNCTIONS:
            symbol = function_symbols.get(self.value, self.value.__name__)
            return f"({str(self.left)} {symbol} {str(self.right)})"
     else:
        return str(self.value)





def choose_function():
    if random.random() < 0.5:
        return random.choice(BINARY_FUNCTIONS)
    else:
        return random.choice(UNARY_FUNCTIONS)
    


def generate_full_tree(depth):
    """Genera un albero con profondità massima (Full method)."""
    if depth == 0:
        # Nodi foglia: terminali
        return Node(random.choice(TERMINALS))
    else:
        func = choose_function()
        if func in BINARY_FUNCTIONS:
            left = generate_full_tree(depth - 1)
            right = generate_full_tree(depth - 1)
            return Node(func, left, right)
        elif func in UNARY_FUNCTIONS:
            child = generate_full_tree(depth - 1)
            return Node(func, child)

def generate_grow_tree(depth):
    """Genera un albero con profondità casuale (Grow method)."""
    if depth == 0 or (random.random() < 0.5):
        # Nodi foglia: terminali
        return Node(random.choice(TERMINALS))
    else:
        func = choose_function()
        if func in BINARY_FUNCTIONS:
            left = generate_grow_tree(depth - 1)
            right = generate_grow_tree(depth - 1)
            return Node(func, left, right)
        elif func in UNARY_FUNCTIONS:
            child = generate_grow_tree(depth - 1)
            return Node(func, child)

def initialize_population(population_size, max_depth):
    """Inizializza la popolazione usando il metodo Ramped Half-and-Half."""
    population = []
    half_size = population_size // 2

    # Metà popolazione con metodo Full
    for _ in range(half_size):
        depth = random.randint(1, max_depth)  # Profondità casuale da 1 a max_depth
        population.append(generate_full_tree(depth))

    # Metà popolazione con metodo Grow
    for _ in range(half_size, population_size):
        depth = random.randint(1, max_depth)
        population.append(generate_grow_tree(depth))

    return population



def mutate(tree, depth):
    if random.random() < 0.3:
        return generate_grow_tree(depth)

    if callable(tree.value):
        if tree.left:
            tree.left = mutate(tree.left, depth - 1)
        if tree.right:
            tree.right = mutate(tree.right, depth - 1)
    return tree

def subtree_mutation(tree):
    if tree is None:
        return None

    if random.random() < 0.3:  ##
        return generate_full_tree(random.randint(1, 3))

    if tree.left:
        tree.left = subtree_mutation(tree.left)
    if tree.right:
        tree.right = subtree_mutation(tree.right)

    return tree

def point_mutation(tree):
    if callable(tree.value):
        if tree.value in UNARY_FUNCTIONS and random.random() < 0.5:
            tree.value = random.choice(UNARY_FUNCTIONS)
        elif tree.value in BINARY_FUNCTIONS and random.random() < 0.5:
            tree.value = random.choice(BINARY_FUNCTIONS)

        if tree.left:
            tree.left = point_mutation(tree.left)
        if tree.right:
            tree.right = point_mutation(tree.right)
    return tree



def crossover(tree1, tree2):
    if random.random() < 0.05:
        return clone_tree(tree2)

    if callable(tree1.value) and callable(tree2.value):
        if tree1.left and tree2.left:
            tree1.left = crossover(tree1.left, tree2.left)
        if tree1.right and tree2.right:
            tree1.right = crossover(tree1.right, tree2.right)
    return tree1



def fitness(tree, x_data, y_data):

    max_depth=3
    predictions = tree.evaluate(x_data)  # Vectorized evaluation
    predictions = np.where(np.isfinite(predictions), predictions, np.inf)
    mse = np.mean((predictions - y_data) ** 2)

    depth_penalty = 0
    if tree_depth(tree) > max_depth:
     depth_penalty = ( mse*0.04 ) * (tree_depth(tree) - max_depth)
    ## 0.3 -> con shape 2 
    # 0.04 -> per problem-3 ( shape 3) -> ma  c'è un po di schifezza .... TODO 
    # 0.02 -> per problem-7 ( shape 2)    --- ( 0.01 + 0.2)

    return mse + depth_penalty


def tree_depth(tree):
    if tree is None or not callable(tree.value):
        return 0
    left_depth = tree_depth(tree.left) if tree.left else 0
    right_depth = tree_depth(tree.right) if tree.right else 0
    return 1 + max(left_depth, right_depth)


def clone_tree(tree):
    if tree is None:
        return None
    return Node(
        tree.value,
        left=clone_tree(tree.left),
        right=clone_tree(tree.right)
    )

def tournament_selection(population, fitness_scores, k):
    candidates = random.sample(list(zip(population, fitness_scores)), k)
    candidates.sort(key=lambda x: x[1])
    return candidates[0][0]



POPULATION_SIZE = 20_000   # num alberi 
GENERATIONS = 50
MAX_DEPTH = 3
ELITE_RATE = 0.2
MUTATION_RATE = 0.7

# provo prima SENZa e poi con EXTINSION 


population = initialize_population(POPULATION_SIZE, MAX_DEPTH)


global_best_tree = None
global_best_fitness = np.inf

for generation in tqdm  (range(GENERATIONS)):

    fitness_scores = np.array([fitness(tree, x_data, y_data) for tree in population])   ####

    # Ordina la popolazione per fitness
    sorted_indices = np.argsort(fitness_scores)
    population = [population[i] for i in sorted_indices]
    fitness_scores = fitness_scores[sorted_indices]


    # Stampa la miglior soluzione della generazione
    print(f"Generazione {generation}: Miglior fitness = {fitness_scores[0]}, Albero = {population[0]}")



    # Elitismo: prendi i migliori individui
    elite_size = int(POPULATION_SIZE * ELITE_RATE)
    new_population = [clone_tree(population[i]) for i in range(elite_size)]

   

    # Riempie il resto della popolazione con crossover e mutazione
    elite_population = population[:elite_size]
    elite_fitness_scores = fitness_scores[:elite_size]

    while len(new_population) < POPULATION_SIZE:
        parent1 = tournament_selection(elite_population, elite_fitness_scores, k=5)
        parent2 = tournament_selection(elite_population, elite_fitness_scores, k=5)
        
        offspring = crossover(clone_tree(parent1), clone_tree(parent2)) # 
       
        if random.random() < MUTATION_RATE: # 

            if random.random() < 0.2:  #  ( 0.2 solo per problem-7), altrimenti 0.4
                offspring = point_mutation(offspring)  
            else:
                offspring = subtree_mutation(offspring)  

        new_population.append(offspring)
    
    population = new_population

    
print("Miglior soluzione trovata")

  0%|          | 0/50 [00:00<?, ?it/s]

/var/folders/09/w7m8wrzs4bddxbzdykd9d6_c0000gn/T/ipykernel_21131/2403755741.py:80: RuntimeWarning: invalid value encountered in power
  result = np.power(safe_left, safe_right)
/var/folders/09/w7m8wrzs4bddxbzdykd9d6_c0000gn/T/ipykernel_21131/2403755741.py:87: RuntimeWarning: invalid value encountered in cos
  result = self.value(left_val)
/var/folders/09/w7m8wrzs4bddxbzdykd9d6_c0000gn/T/ipykernel_21131/2403755741.py:87: RuntimeWarning: invalid value encountered in tan
  result = self.value(left_val)
/var/folders/09/w7m8wrzs4bddxbzdykd9d6_c0000gn/T/ipykernel_21131/2403755741.py:87: RuntimeWarning: invalid value encountered in sin
  result = self.value(left_val)
/var/folders/09/w7m8wrzs4bddxbzdykd9d6_c0000gn/T/ipykernel_21131/2403755741.py:92: RuntimeWarning: invalid value encountered in multiply
  result = self.value(left_val, right_val)
/var/folders/09/w7m8wrzs4bddxbzdykd9d6_c0000gn/T/ipykernel_21131/2403755741.py:92: RuntimeWarning: invalid value encountered in subtract
  result = sel

Generazione 0: Miglior fitness = 28500530991170.965, Albero = (x0 ^ 9)


/var/folders/09/w7m8wrzs4bddxbzdykd9d6_c0000gn/T/ipykernel_21131/2403755741.py:80: RuntimeWarning: overflow encountered in power
  result = np.power(safe_left, safe_right)


Generazione 1: Miglior fitness = 27050449085073.59, Albero = ((6 - 2) ^ (4 ^ x0))


/var/folders/09/w7m8wrzs4bddxbzdykd9d6_c0000gn/T/ipykernel_21131/2403755741.py:275: RuntimeWarning: overflow encountered in square
  mse = np.mean((predictions - y_data) ** 2)


Generazione 2: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 3: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 4: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 5: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 6: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 7: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 8: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 9: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 10: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 11: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 12: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 13: Miglior fitness = 19065119581188.062, Albero = (x0 * exp((9 + 5)))
Generazione 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x103b85be0>>
Traceback (most recent call last):
  File "/Users/alessandrocarlone/Desktop/CI-projects/CI-2024/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
